In [26]:
import pandas as pd

data=pd.read_csv(r"recipes.csv")

#Filtering columns based on requirement :-
cols_filtered_data=data[["RecipeId", "Name", "AuthorId", "AuthorName", "CookTime", "PrepTime",
        "DatePublished", "RecipeCategory", "AggregatedRating", "ReviewCount", "RecipeIngredientParts",
        "Calories", "FatContent", "SaturatedFatContent", "CholesterolContent", "SodiumContent",
        "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent", "RecipeServings"]]

<ipython-input-26-10860cbd468d>:3: DtypeWarning: Columns (0,2,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r"recipes.csv")


In [27]:
#removing trash rows
cols_filtered_data=cols_filtered_data.drop(index=range(1438, 1488))

In [29]:



cols_filtered_data["ReviewCount"] = pd.to_numeric(cols_filtered_data["ReviewCount"], errors="coerce")


cols_filtered_data = cols_filtered_data.dropna(subset=["ReviewCount"])

cols_filtered_data = cols_filtered_data[cols_filtered_data["ReviewCount"] > 3]



In [31]:
import pandas as pd

# Replace non-numeric AggregatedRating with a default value (e.g., 0)
cols_filtered_data["AggregatedRating"] = pd.to_numeric(
    cols_filtered_data["AggregatedRating"], errors="coerce"
).fillna(0)

# Choosing mixed rating dishes
rating_above_45 = cols_filtered_data[cols_filtered_data["AggregatedRating"] >= 4.5][0:10000]
rating_below_4 = cols_filtered_data[cols_filtered_data["AggregatedRating"] < 4.5][0:33000]

# Concatenating the filtered datasets
cols_filtered_data = pd.concat([rating_above_45, rating_below_4], axis=0)


print(cols_filtered_data)



       RecipeId                                               Name  \
0            38                  Low-Fat Berry Blue Frozen Dessert   
2            40                                      Best Lemonade   
4            42                                       Cabbage Soup   
6            44                             Warm Chicken A La King   
11           49                           Chicken Breasts Lombardi   
...         ...                                                ...   
518048   535783                    Cheesesteak Stuffed Onion Rings   
518943   536688                         COCO OATMEAL HONEY COOKIES   
519094   536843  Sheet Pan Turkey Caprese Meatballs With Rosema...   
519181   536930                      Cheesy Buffalo Chicken Skulls   
520453   538222               Copycat Jimmy Dean Breakfast Sausage   

          AuthorId         AuthorName CookTime PrepTime         DatePublished  \
0             1533             Dancer    PT24H    PT45M  1999-08-09T21:46:00Z 

In [32]:
#Modifying column to store date in required format :- (Only year of publish)
cols_filtered_data["DatePublished"]=cols_filtered_data["DatePublished"].str[0:4]

In [33]:

#Replacing all blank entries in columns with default data
for i in cols_filtered_data:
    if i not in ["RecipeId", "Name", "AuthorId", "AuthorName","DatePublished", "RecipeCategory","RecipeServings"]:
        cols_filtered_data[i].fillna(0, inplace=True)
    elif i=="RecipeServings":
        cols_filtered_data[i].fillna(-1, inplace=True)   #-1 code for serving parameter not applicable. Eg: items like sauce

<ipython-input-33-ef685f07f131>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cols_filtered_data[i].fillna(0, inplace=True)
<ipython-input-33-ef685f07f131>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [34]:
def ingredient_format(strig):
    if strig[0:2]!="c(":
        return "c("+strig+")"
    else:
        return strig

In [35]:
cols_filtered_data["RecipeIngredientParts"]=cols_filtered_data["RecipeIngredientParts"].apply(ingredient_format)

In [36]:
def time_to_mins(strig):
    if strig==0:
        return 0
    else:
        a=0
        b=0
        if strig[1]=="H":
            a=int(strig[0])
        elif len(strig)>2 and strig[2]=="H":
            a=int(strig[0:2])
        if strig[-1]=="M":
            if len(strig)>2 and strig[-3].isdigit():
                b=int(strig[-3:-1])
            else:
                b=int(strig[-2])
        return a*60+b

In [37]:
#Converting PrepTime and CookTime column entries to numerical format :-
cols_filtered_data["PrepTime"]=cols_filtered_data["PrepTime"].str.lstrip("PT")
cols_filtered_data["CookTime"]=cols_filtered_data["CookTime"].str.lstrip("PT")
cols_filtered_data["PrepTime"].fillna(0,inplace=True)
cols_filtered_data["CookTime"].fillna(0,inplace=True)
cols_filtered_data["PrepTime"].replace("0S",0,inplace=True)
cols_filtered_data["CookTime"].replace("0S",0,inplace=True)
cols_filtered_data["PrepTime"]=cols_filtered_data["PrepTime"].apply(time_to_mins)
cols_filtered_data["CookTime"]=cols_filtered_data["CookTime"].apply(time_to_mins)

<ipython-input-37-7a37ad33a951>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cols_filtered_data["PrepTime"].fillna(0,inplace=True)
<ipython-input-37-7a37ad33a951>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [38]:

#Removing duplicate items :-
final_recipes_dataset=cols_filtered_data.drop_duplicates(subset=["Name"])

In [39]:
data=pd.read_csv(r"reviews.csv")

#Filtering columns based on requirement :-
recipes_dataset=data[["ReviewId","RecipeId", "AuthorId", "AuthorName", "Rating"]]
recipes_dataset=recipes_dataset.rename(columns={"AuthorId":"ReviewerID", "AuthorName":"ReviewerName"})


<ipython-input-39-40f61d61001b>:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r"reviews.csv")


In [40]:
final_processed_dataset=pd.merge(final_recipes_dataset,recipes_dataset,on="RecipeId")

In [41]:
reviewer_review_counts=final_processed_dataset["ReviewerName"].value_counts()
final_processed_dataset=final_processed_dataset[final_processed_dataset["ReviewerName"].isin(reviewer_review_counts[reviewer_review_counts>5].index)]

In [42]:
final_processed_dataset.to_csv(r"Final_Processed_Dataset.csv",index=False)